# Extracting Audio Features with Essentia Streaming

Essentia is an open-source C++ library for audio analysis and audio-based music information retrieval.
Documentation: http://essentia.upf.edu/documentation.html

In [8]:
import sys
sys.path.append('../')

import os
import csv
import soundfile as sf
import essentia
import essentia.standard as esstd
import essentia.streaming as esstr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fnmatch import fnmatch
from pathlib import Path

In [9]:
DATA_DIR = Path('../data/raw/')
MODELS_DIR = Path('../models/')
RESULTS_DIR = Path('../data/features/')
PLOTS_DIR = Path('../docs/plots/')

In [10]:
# Retrieve wav files
def retrieve_wav_files(directory):
    """
    Retrieve all the files in a directory and its subdirectories
    :param directory: the directory to search
    :return: a list of files
    """
    wav_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))

    wav_files.sort()
    return wav_files

In [11]:
def extract_features_essentia(data, sample_rate):
    """Extract various audio features using Essentia's streaming mode."""
    
    frame_size = 1024
    features = {}
    pool = essentia.Pool()

    # Instantiate the algorithms
    loader = esstr.VectorInput(data)
    fcut = esstr.FrameCutter(frameSize=frame_size, hopSize=512)
    w = esstr.Windowing()
    spec = esstr.Spectrum(size=frame_size)

    gain = esstr.ReplayGain(sampleRate=sample_rate)
    leq = esstr.Leq()
    loudness = esstr.Loudness()
    zero_crossing_rate = esstr.ZeroCrossingRate()
    centroid = esstr.SpectralCentroidTime(sampleRate=sample_rate)
    pitch = esstr.PitchYin(sampleRate=sample_rate, frameSize=frame_size)
    
    # New features' algorithms
    flatness = esstr.FlatnessDB()
    hfc = esstr.HFC(sampleRate=sample_rate)
    
    # Connect the algorithms
    loader.data >> fcut.signal
    loader.data >> gain.signal
    loader.data >> leq.signal

    fcut.frame >> centroid.array
    fcut.frame >> loudness.signal
    fcut.frame >> zero_crossing_rate.signal
    fcut.frame >> pitch.signal
    
    fcut.frame >> w.frame
    w.frame >> spec.frame

    spec.spectrum >> flatness.array
    spec.spectrum >> hfc.spectrum
    
    # Add the features to the pool
    gain.replayGain >> (pool, 'gain')
    leq.leq >> (pool, 'leq')
    loudness.loudness >> (pool, 'loudness')
    zero_crossing_rate.zeroCrossingRate >> (pool, 'zcr')
    centroid.centroid >> (pool, 'centroid')
    pitch.pitch >> (pool, 'pitch')
    pitch.pitchConfidence >> (pool, 'confidence')
    
    flatness.flatnessDB >> (pool, 'flatness')
    hfc.hfc >> (pool, 'hfc')
    
    # Run the network
    essentia.run(loader)

    aggrpool = esstd.PoolAggregator(defaultStats = ["mean"])(pool)    

    # Extract features
    for feature in ['gain', 'leq', 'loudness.mean', 'zcr.mean', 'pitch.mean', 'confidence.mean', 'flatness.mean', 'hfc.mean']:
        value = np.array(aggrpool[feature]).flatten()[0]
        # Remove the .mean suffix for storage
        feature_name = feature.replace('.mean', '')

        if feature_name not in features:
            features[feature_name] = []
        features[feature_name].append(value)

    return features

## EGFxSet

**This may take long time...**

In [12]:
dry_samples_path = os.path.join(DATA_DIR, "egfxset/Clean")
wet_samples_path = os.path.join(DATA_DIR, "egfxset/Spring Reverb")

# Make file lists
dry_files = retrieve_wav_files(dry_samples_path)
wet_files = retrieve_wav_files(wet_samples_path)

print(f"Number of dry samples: {len(dry_files)}")
print(f"Number of wet samples: {len(wet_files)}")

Number of dry samples: 690
Number of wet samples: 690


In [13]:
for dataset, files in [("dry", dry_files), ("wet", wet_files)]:
    for file in files:
        data, sample_rate = sf.read(file, dtype='float32')
        features = extract_features_essentia(data, sample_rate)
        
        # Add a 'file' key to the features dictionary to identify the source file
        relative_path = Path(file).relative_to(DATA_DIR)
        features['file'] = str(relative_path)
        
        # Define the output CSV filename for this file
        output_csv_filename = Path(RESULTS_DIR) / f"egfxset_{dataset}_essentia.csv"
        
        # Check if the CSV file exists, if not, write the header
        if not output_csv_filename.exists():
            with open(output_csv_filename, 'w', newline='') as csvfile:
                fieldnames = features.keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
        
        # Remove square brackets from feature values
        for key, value in features.items():
            if isinstance(value, list):
                features[key] = value[0]
        
        # Write the feature data for this file
        with open(output_csv_filename, 'a', newline='') as csvfile:
            fieldnames = features.keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Check if the file is empty (no header written) and write the header if needed
            if os.stat(output_csv_filename).st_size == 0:
                writer.writeheader()
            
            writer.writerow(features)

        print(f"Saved {dataset} feature file for {file} to {output_csv_filename}")


Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-0.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-1.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-10.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-11.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-12.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-13.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-14.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data/raw/egfxset/Clean/Bridge-Middle/1-15.wav to ../data/features/egfxset_dry_essentia.csv
Saved dry feature file for ../data